In [1]:
import pandas as pd
import numpy as np
import time
import requests

### Grabs the players data using the fetchplayerstats class
Enter the year you want in season and if you want Regualr Season or Playoffs in season_type

In [ ]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2024-25', season_type='Playoffs', sleep_time=1, max_workers=5)
data['OPP_TEAM_ID'] = data['OPP_TEAM_ID'].astype(int)
data.to_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')

In [5]:
pd.set_option('display.max_columns', None)
# data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')
data = pd.read_csv('REGULAR_DATA/SEASON_25.csv')
data.drop(columns=['Unnamed: 0'], inplace=True)
data

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,COMMENT,OFF_RATING,E_OFF_RATING,DEF_RATING,E_DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,EFG_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,POSS,PACE_PER40,E_USG_PCT,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE
0,Al Horford,201143,BOS vs. NYK,BOS,1610612738,NYK,1,22400061,2024-10-22,W,26,11,5,3,4,7,0.571,3,5,0.600,0,0,NaN,0,3,1,1,0,2,19,28.1,1.571,0.785714,C,NaN,154.9,158.8,117.6,116.7,37.3,0.000,0.158,0.068,0.200,0.786,0.00,0.117,0.786,93.08,93.85,0.108,51,78.21,0.121,"OCT 22, 2024",240,48,95,0.505,29,61,0.475,7,8,0.875,11,29,40,33,6,3,3,15,132,145.824127,2.0,3.0,34.0,0.551282,1610612752,145.824127,90.52,90.78,91.04
1,Dalton Knecht,1642261,LAL vs. MIN,LAL,1610612747,MIN,1,22400062,2024-10-22,W,16,5,2,1,2,4,0.500,1,3,0.333,0,0,NaN,0,1,1,0,1,1,7,11.2,1.250,0.625000,NaN,NaN,129.4,128.7,115.6,112.3,13.8,0.000,0.063,0.031,0.118,0.625,2.00,0.122,0.625,102.12,100.36,0.070,34,83.63,0.128,"OCT 22, 2024",240,42,95,0.442,5,30,0.167,21,25,0.840,15,31,46,22,7,8,7,22,110,112.244898,4.0,1.0,47.0,0.411765,1610612750,112.244898,98.00,98.94,99.88
2,Jaden McDaniels,1630183,MIN @ LAL,MIN,1610612750,LAL,0,22400062,2024-10-22,L,16,6,1,2,3,8,0.375,0,3,0.000,0,0,NaN,0,2,1,0,1,5,-8,11.9,0.750,0.375000,F,NaN,90.0,80.0,116.7,116.4,-26.7,0.000,0.105,0.059,0.143,0.375,1.00,0.243,0.375,95.76,90.00,-0.022,30,75.00,0.245,"OCT 22, 2024",240,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,15,22,103,103.123748,7.0,8.0,46.0,0.442105,1610612747,103.123748,99.88,98.94,98.00
3,Naz Reid,1629675,MIN @ LAL,MIN,1610612750,LAL,0,22400062,2024-10-22,L,26,12,1,4,3,8,0.375,2,4,0.500,4,4,1.0,1,3,0,0,1,3,-6,17.3,1.230,0.500000,NaN,NaN,115.1,112.9,124.1,118.4,-9.0,0.034,0.107,0.070,0.056,0.500,1.00,0.172,0.615,100.77,97.46,0.074,53,81.21,0.174,"OCT 22, 2024",240,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,15,22,103,103.123748,7.0,8.0,46.0,0.442105,1610612747,103.123748,99.88,98.94,98.00
4,Nickeil Alexander-Walker,1629638,MIN @ LAL,MIN,1610612750,LAL,0,22400062,2024-10-22,L,28,14,0,5,5,7,0.714,3,4,0.750,1,2,0.5,2,3,0,0,2,0,13,18.0,1.777,0.928571,NaN,NaN,135.2,130.8,107.1,105.2,28.0,0.063,0.097,0.079,0.000,0.929,0.00,0.147,0.888,95.47,93.07,0.124,54,77.56,0.152,"OCT 22, 2024",240,35,85,0.412,13,41,0.317,20,27,0.741,12,35,47,17,4,1,15,22,103,103.123748,7.0,8.0,46.0,0.442105,1610612747,103.123748,99.88,98.94,98.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26301,Caleb Martin,1628997,DAL @ MEM,DAL,1610612742,MEM,0,22401194,2025-04-13,L,21,6,0,3,3,8,0.375,0,3,0.000,0,0,NaN,1,2,0,0,0,2,-13,9.6,0.750,0.375000,NaN,NaN,95.8,96.4,125.5,129.6,-29.7,0.037,0.087,0.060,0.000,0.375,0.00,0.151,0.375,104.24,106.21,0.015,48,88.51,0.155,"APR 13, 2025",240,38,87,0.437,5,28,0.179,16,22,0.727,10,28,38,20,6,3,18,13,97,92.663355,13.0,7.0,53.0,0.500000,1610612763,92.663355,104.68,103.64,102.60
26302,Dwight Powell,203939,DAL @ MEM,DAL,1610612742,MEM,0,22401194,2025-04-13,L,16,6,0,0,2,4,0.500,1,2,0.500,1,2,0.5,0,0,1,0,0,1,-12,9.0,1.230,0.625000,NaN,NaN,90.0,87.8,130.0,128.1,-40.0,0.000,0.000,0.000,0.000,0.625,0.00,0.152,0.615,92.96,91.14,0.050,30,75.95,0.157,"APR 13, 2025",240,38,87,0.437,5,28,0.179,16,22,0.727,10,28,38,20,6,3,18,13,97,92.663355,13.0,7.0,53.0,0.500000,1610612763,92.663355,1

In [4]:
from NBAData.features import *
from NBAData.playerPositions import *

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = calculate_rolling_averages(data, [2, 4, 6])
data = home_away_averages(data)
data = calculate_days_of_rest(data)
data = preprocessGamesData(data)
data = addLagFeatures(data)
data = add_all_opponent_features(data)
data = assign_position_with_cache(
    data, 
    cache_file='nba_positions.csv',
    max_workers=4, 
    delay_between_requests=1.5
)
data.to_csv('REGULAR_DATA/SEASON_25_FEATURES.csv')

Loading position cache...
Loaded 992 players from cache
Found 219 unique players, 0 need to be fetched
Applying positions to dataset...
Creating position flags...
Position assignment completed!


## ASSIST data

In [ ]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2024-25', season_type='Regular Season', sleep_time=1, max_workers=5)
data.to_csv('REGULAR_DATA/SEASON_25.csv')

In [ ]:
from NBAData.features import *
from NBAData.playerPositions import *

data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = calculate_rolling_averages_ast(data, [2, 4, 6])
data = home_away_ast_averages(data)
data = calculate_days_of_rest(data)
data = preprocessGamesData(data)
data = addLagFeatures_Ast(data)
data = add_all_opponent_features_ast(data)
data = assign_position_with_cache(
    data, 
    cache_file='nba_positions.csv',
    max_workers=4, 
    delay_between_requests=1.5
)
data.to_csv('PLAYOFF_DATA/PLAYOFFS_25_REB_FEATURES.csv')

Loading position cache...
Loaded 992 players from cache
Found 219 unique players, 0 need to be fetched
Applying positions to dataset...
Creating position flags...
Position assignment completed!


## REBOUND Data

In [ ]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2022-23', season_type='Playoffs', sleep_time=1, max_workers=5)
data.to_csv('PLAYOFF_DATA/PLAYOFFS_23.csv')

In [11]:
from NBAData.features import *
from NBAData.playerPositions import *

data = pd.read_csv('REGULAR_DATA/SEASON_25.csv')

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = calculate_rolling_averages_reb(data, [2, 4, 6])
data = home_away_reb_averages(data)
data = calculate_days_of_rest(data)
data = preprocessGamesData(data)
data = addLagFeatures_Reb(data)
data = add_all_opponent_features_reb(data)
data = assign_position_with_cache(
    data, 
    cache_file='nba_positions.csv',
    max_workers=4, 
    delay_between_requests=1.5
)
data.to_csv('REGULAR_DATA/SEASON_25_REB_FEATURES.csv')

Loading position cache...
Loaded 992 players from cache
Found 569 unique players, 0 need to be fetched
Applying positions to dataset...
Creating position flags...
Position assignment completed!
